## Tugas Praktikum

1. Buatlah model SVM dengan menggunakan data voice.csv dengan ketentuan,

  - Split data dengan rasio 70:30 dan 80:20 untuk setiap model yang akan dibangun.
    - Gunakan model dengan kernel linier.
    - Gunakan model dengan kernel polynomial.
    - Gunakan model dengan kernel RBF.

  - Tabulasikan performansi setiap split dan kernel berdasarkan metrik akurasi.

2. Gunakan data pada praktikum 5 untuk membuat model klasifikasi siang dan malam menggunakan SVM dengan kernel RBF menggunakan fitur histrogram. Gunakan rasio 80:20. Anda dapat bereksperimen dengan hyperparameter tunning dari kernel RBF. Catat performansi akurasinya!

In [ ]:
# SVM untuk klasifikasi suara (voice.csv) – versi ditulis ulang
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

file_path = Path("/content/voice.csv")
if not file_path.is_file():
    raise FileNotFoundError(f"File voice.csv tidak ditemukan pada lokasi: {file_path.resolve()}")

df = pd.read_csv(file_path)

if "label" not in df.columns:
    raise KeyError("Kolom label tidak ditemukan. Pastikan dataset memiliki kolom bernama 'label'.")

# Memastikan kolom fitur numerik
feature_columns = [col for col in df.columns if col != "label"]
df[feature_columns] = df[feature_columns].apply(pd.to_numeric, errors="coerce")

# Menghapus data yang rusak / NaN
df = df.dropna()

# Konversi label male/female menjadi 0/1
label_dict = {"male": 0, "female": 1}
df["label"] = df["label"].map(label_dict)

if df["label"].isna().any():
    raise ValueError("Ditemukan label selain male/female. Silakan periksa isi dataset.")

X_data = df[feature_columns].to_numpy()
y_data = df["label"].astype(int).to_numpy()

tes_sizes = [0.30, 0.20]    # untuk 70:30 dan 80:20
kernel_ops = [
    ("linear", {"kernel": "linear", "random_state": 42}),
    ("poly",   {"kernel": "poly",   "degree": 3, "random_state": 42}),
    ("rbf",    {"kernel": "rbf",                "random_state": 42}),
]

hasil = []

for t in tes_sizes:

    X_latih, X_uji, y_latih, y_uji = train_test_split(
        X_data,
        y_data,
        test_size=t,
        random_state=42,
        stratify=y_data
    )

    rasio_str = f"{int((1-t)*100)}:{int(t*100)}"

    for nama_kernel, args_svc in kernel_ops:

        # Pipeline scaler + SVC
        model = Pipeline([
            ("scaler", StandardScaler()),
            ("svm", SVC(**args_svc))
        ])

        model.fit(X_latih, y_latih)

        akurasi_train = accuracy_score(y_latih, model.predict(X_latih))
        akurasi_test  = accuracy_score(y_uji, model.predict(X_uji))

        hasil.append({
            "split": rasio_str,
            "kernel": nama_kernel,
            "acc_train": akurasi_train,
            "acc_test": akurasi_test
        })

hasil_df = pd.DataFrame(hasil).sort_values(["split", "kernel"]).reset_index(drop=True)

print("\n== Rekap Hasil Percobaan SVM ==")
print(hasil_df)

# Pivot untuk perbandingan cepat akurasi test
try:
    print("\nPivot akurasi (test accuracy):")
    print(hasil_df.pivot(index="kernel", columns="split", values="acc_test"))
except:
    pass



== Rekap Hasil Percobaan SVM ==
   split  kernel  acc_train  acc_test
0  70:30  linear   0.977898  0.969506
1  70:30    poly   0.967975  0.949527
2  70:30     rbf   0.984664  0.981073
3  80:20  linear   0.977901  0.968454
4  80:20    poly   0.968035  0.952681
5  80:20     rbf   0.986188  0.979495

Pivot akurasi (test accuracy):
split      70:30     80:20
kernel                    
linear  0.969506  0.968454
poly    0.949527  0.952681
rbf     0.981073  0.979495
